# Challenge Python L1 

|  Region | City Name |  Languaje | Time  |
|---|---|---|---|
|  Africa | Angola  |  AF4F4762F9BD3F0F4A10CAF5B6E63DC4CE543724 | 0.23 ms  |
|   |   |   |   |
|   |   |   |   |

Desarrolle una aplicacion en python que genere la tabla anterior teniendo las siguientes consideraciones:

1. De https://rapidapi.com/apilayernet/api/rest-countries-v1, obtenga todas las regiones existentes.
2. De https://restcountries.eu/ Obtenga un pais por region apartir de la region optenida del punto 1.
3. De https://restcountries.eu/ obtenga el nombre del idioma que habla el pais y encriptelo con SHA1
4. En la columna Time ponga el tiempo que tardo en armar la fila (debe ser automatico)
5. La tabla debe ser creada en un DataFrame con la libreria PANDAS
6. Con funciones de la libreria pandas muestre el tiempo total, el tiempo promedio, el tiempo minimo y el maximo que tardo en procesar toda las filas de la tabla.
7. Guarde el resultado en sqlite.
8. Genere un Json de la tabla creada y guardelo como data.json
9. La prueba debe ser entregada en un repositorio git.


**Es un plus si:**
* No usa famework
* Entrega Test Unitarios
* Presenta un diseño de su solucion.


In [1]:
# imports
import pandas as pd
import requests
from time import time
import hashlib
from sqlalchemy import create_engine

# Utility

In [2]:

def get_api_to_df(url, headers):
    """
    Cargar datos del api en un df
    """
    response = requests.request("GET", url, headers=headers)
    return pd.DataFrame(response.json())

        

def count_elapsed_time(region):
    """
    count elapsed time to add a row
    """
    # Start counting.
    start_time = time()
    # make row
    url = "https://restcountries.eu/rest/v2/region/" + region.lower() + "?fields=name;languages"
    first_country_by_region = get_api_to_df(url, {}).head(1)
    # Calculate the elapsed time.
    first_country_by_region["Time"] = time() - start_time
    first_country_by_region["Region"] = region
    
    return first_country_by_region


1. De https://rapidapi.com/apilayernet/api/rest-countries-v1, obtenga todas las regiones existentes.

In [3]:

headers_rapidapi = {
    'x-rapidapi-key': "dbe2e72308msh570465d7bfe30fap1f97e9jsnc358b8a5def7",
    'x-rapidapi-host': "restcountries-v1.p.rapidapi.com"
    }

df_regions = get_api_to_df("https://restcountries-v1.p.rapidapi.com/all?fields=region",headers_rapidapi)["region"].unique()
df_regions = df_regions[df_regions != '']
print(df_regions)

['Asia' 'Europe' 'Africa' 'Oceania' 'Americas' 'Polar']


2. De https://restcountries.eu/ Obtenga un pais por region apartir de la region optenida del punto 1.
4. En la columna Time ponga el tiempo que tardo en armar la fila (debe ser automatico)
5. La tabla debe ser creada en un DataFrame con la libreria PANDAS

In [4]:
df_countries = pd.DataFrame()

for region in df_regions:
    first_country_by_region = count_elapsed_time(region).head(1)
    df_countries = pd.concat([df_countries,first_country_by_region])

df_countries.rename(columns={"languages": "Languages", "name": "City Name"},inplace = True)
df_countries

,Languages,City Name,Time,Region
0,"[{'iso639_1': 'ps', 'iso639_2': 'pus', 'name':...",Afghanistan,0.419878,Asia
0,"[{'iso639_1': 'sv', 'iso639_2': 'swe', 'name':...",Åland Islands,0.289528,Europe
0,"[{'iso639_1': 'ar', 'iso639_2': 'ara', 'name':...",Algeria,0.307405,Africa
0,"[{'iso639_1': 'en', 'iso639_2': 'eng', 'name':...",American Samoa,0.444539,Oceania
0,"[{'iso639_1': 'en', 'iso639_2': 'eng', 'name':...",Anguilla,0.284105,Americas
0,"[{'iso639_1': 'en', 'iso639_2': 'eng', 'name':...",Antarctica,0.281040,Polar


3. De https://restcountries.eu/ obtenga el nombre del idioma que habla el pais y encriptelo con SHA1

In [5]:
df_countries['Languages'] = [hashlib.sha1(str.encode(str(i))).hexdigest().upper() for i in df_countries['Languages']]
df_countries

,Languages,City Name,Time,Region
0,EAEC9694B6F77900441DCF5C40E68BA6AA012859,Afghanistan,0.419878,Asia
0,1FF639F07C4D48DC20CD42F806D7C08A421E316A,Åland Islands,0.289528,Europe
0,F937A1DD22E17FED2E633E04C2C116D037596326,Algeria,0.307405,Africa
0,532EBB6AD7726E1A1CC6FD7906A601CF1411F944,American Samoa,0.444539,Oceania
0,D763659922F4C5348B508D92346C6AA6901A9440,Anguilla,0.284105,Americas
0,5EE1CB5FFB3CC7378100144C1633BA990E7B6924,Antarctica,0.281040,Polar


6. Con funciones de la libreria pandas muestre el tiempo total, el tiempo promedio, el tiempo minimo y el maximo que tardo en procesar toda las filas de la tabla.

In [6]:
df_countries['Time'].describe()

count    6.000000
mean     0.337749
std      0.074147
min      0.281040
25%      0.285461
50%      0.298467
75%      0.391760
max      0.444539
Name: Time, dtype: float64

7. Guarde el resultado en sqlite.

In [7]:
engine = create_engine('sqlite://', echo=False)
df_countries.to_sql('countries', con=engine)
engine.execute("SELECT * FROM countries").fetchall()

[(0, 'EAEC9694B6F77900441DCF5C40E68BA6AA012859', 'Afghanistan', 0.4198777675628662, 'Asia'),
 (0, '1FF639F07C4D48DC20CD42F806D7C08A421E316A', 'Åland Islands', 0.28952789306640625, 'Europe'),
 (0, 'F937A1DD22E17FED2E633E04C2C116D037596326', 'Algeria', 0.3074054718017578, 'Africa'),
 (0, '532EBB6AD7726E1A1CC6FD7906A601CF1411F944', 'American Samoa', 0.44453907012939453, 'Oceania'),
 (0, 'D763659922F4C5348B508D92346C6AA6901A9440', 'Anguilla', 0.2841048240661621, 'Americas'),
 (0, '5EE1CB5FFB3CC7378100144C1633BA990E7B6924', 'Antarctica', 0.2810401916503906, 'Polar')]

8. Genere un Json de la tabla creada y guardelo como data.json

In [8]:
df_countries.to_json("data.json",orient='records')

Cesar Muricio Dueñas Ruiz

# Enjoy!! =)